In [68]:
%matplotlib inline
import time

# python 3
import urllib.error
from urllib.request import urlopen
import datetime
#from itertools import ifilter
from collections import Counter, defaultdict
import xml.etree.ElementTree as ET

from bs4 import BeautifulSoup
import matplotlib.pylab as plt
import pandas as pd
import numpy as np
#import bibtexparser

pd.set_option('mode.chained_assignment','warn')

OAI = "{http://www.openarchives.org/OAI/2.0/}"
ARXIV = "{http://arxiv.org/OAI/arXiv/}"
URL = "http://export.arxiv.org/oai2?verb=ListRecords&from=2016-08-01&until=2016-08-05&metadataPrefix=arXiv&set=physics:astro-ph"

target_name_dict = { 'astro-ph.GA' : 1,
                     'astro-ph.SR' : 2,
                     'astro-ph.IM' : 3,
                     'astro-ph.EP' : 4,
                     'astro-ph.SR' : 5,
                     'astro-ph.HE' : 6,
                     'astro-ph.CO' : 7
                   }

In [8]:
url = URL

try:
    response = urlopen(url)

except urllib.error.HTTPError as e:
    if e.code == 503:
        to = int(e.hdrs.get("retry-after", 30))
        print("Got 503. Retrying after {0:d} seconds.".format(to))

        time.sleep(to)
#        continue

    else:
        raise

xml = response.read()

root = ET.fromstring(xml)

for record in root.find(OAI+'ListRecords').findall(OAI+"record"):
    arxiv_id = record.find(OAI+'header').find(OAI+'identifier')
    meta = record.find(OAI+'metadata')
    info = meta.find(ARXIV+"arXiv")
    created = info.find(ARXIV+"created").text
    created = datetime.datetime.strptime(created, "%Y-%m-%d")
    categories = info.find(ARXIV+"categories").text

    # if there is more than one DOI use the first one
    # often the second one (if it exists at all) refers
    # to an eratum or similar
    doi = info.find(ARXIV+"doi")
    if doi is not None:
        doi = doi.text.split()[0]

    contents = {'title': info.find(ARXIV+"title").text,
                'id': info.find(ARXIV+"id").text,#arxiv_id.text[4:],
                'abstract': info.find(ARXIV+"abstract").text.strip(),
                'created': created,
                'categories': categories.split(),
                'doi': doi,
                }
    print(categories)

astro-ph.GA
astro-ph.SR
astro-ph.SR physics.comp-ph
astro-ph.SR physics.comp-ph
astro-ph.SR physics.comp-ph
astro-ph.SR physics.comp-ph
hep-th astro-ph.CO
astro-ph.GA astro-ph.CO
astro-ph.SR
astro-ph.SR
astro-ph.GA astro-ph.CO
astro-ph.IM astro-ph.HE physics.ins-det
physics.comp-ph astro-ph.SR cs.OH
astro-ph.GA
astro-ph.HE
hep-ex astro-ph.HE
astro-ph.SR physics.space-ph
astro-ph.GA
astro-ph.CO
astro-ph.GA astro-ph.CO
gr-qc astro-ph.CO math-ph math.CA math.DS math.MP
astro-ph.CO astro-ph.HE
astro-ph.CO
astro-ph.IM astro-ph.GA physics.chem-ph
gr-qc astro-ph.HE
astro-ph.GA
astro-ph.GA astro-ph.IM
astro-ph.GA
gr-qc astro-ph.CO
astro-ph.HE
hep-th astro-ph.CO
astro-ph.HE
astro-ph.CO stat.AP
gr-qc astro-ph.HE hep-th
astro-ph.SR nlin.CD
gr-qc astro-ph.CO hep-ph hep-th
astro-ph.IM astro-ph.HE gr-qc
hep-ph astro-ph.CO gr-qc
astro-ph.HE
astro-ph.HE astro-ph.CO astro-ph.IM hep-ph
astro-ph.GA astro-ph.CO
astro-ph.CO
astro-ph.HE
hep-th astro-ph.CO gr-qc hep-ph
astro-ph.HE astro-ph.SR
astro-ph.IM ast

In [7]:
categories

'astro-ph.HE'

In [107]:
store = pd.HDFStore("astroph_Aug2016.h5")
df = store['df']
store.close()
df['label'] = pd.Series(dtype='int64')

In [112]:
for i, rows in df.iterrows():
    cat = df.get_value(i, 'categories')
    if any(['astro-ph.' in c for c in cat]):
        for j in range(len(cat)):
            if cat[j] in target_name_dict.keys():
                label = target_name_dict[cat[j]]
                print(i, label, cat)
                df.set_value(i, 'label', label)
                break
    else:
        df.drop(i, inplace=True)

23 1 ['physics.hist-ph', 'astro-ph', 'astro-ph.GA']
31 5 ['astro-ph.SR']
32 1 ['astro-ph.GA', 'astro-ph.SR']
33 7 ['astro-ph.CO', 'astro-ph.GA']
34 7 ['astro-ph.CO', 'astro-ph.HE']
35 1 ['astro-ph.GA']
36 7 ['astro-ph.CO', 'gr-qc']
37 1 ['astro-ph.GA']
38 6 ['astro-ph.HE']
39 6 ['astro-ph.HE', 'astro-ph.CO']
40 7 ['gr-qc', 'astro-ph.CO', 'hep-th', 'math-ph', 'math.MP']
41 5 ['astro-ph.SR']
42 7 ['astro-ph.CO', 'astro-ph.GA']
43 7 ['astro-ph.CO', 'astro-ph.GA']
44 5 ['astro-ph.SR', 'astro-ph.EP']
45 1 ['astro-ph.GA', 'astro-ph.SR']
46 6 ['astro-ph.HE']
47 7 ['astro-ph.CO']
48 7 ['astro-ph.CO', 'gr-qc']
49 5 ['astro-ph.SR']
50 1 ['astro-ph.GA']
51 1 ['astro-ph.GA']
52 1 ['astro-ph.GA']
53 1 ['astro-ph.GA']
54 5 ['astro-ph.SR']
55 1 ['astro-ph.GA']
56 5 ['astro-ph.SR', 'astro-ph.GA', 'astro-ph.IM']
57 3 ['astro-ph.IM', 'astro-ph.GA', 'astro-ph.HE']
58 5 ['astro-ph.SR']
59 1 ['astro-ph.GA']
60 5 ['astro-ph.SR']
61 6 ['astro-ph.HE', 'astro-ph.GA', 'physics.comp-ph']
62 7 ['astro-ph.CO']
63 

482 4 ['astro-ph.EP']
483 7 ['astro-ph.CO']
484 7 ['astro-ph.CO', 'astro-ph.GA']
485 7 ['gr-qc', 'astro-ph.CO']
486 4 ['astro-ph.EP']
487 6 ['astro-ph.HE']
488 7 ['astro-ph.CO', 'hep-ph', 'hep-th']
489 6 ['astro-ph.HE', 'astro-ph.SR']
490 3 ['astro-ph.IM', 'astro-ph.EP']
491 6 ['astro-ph.HE', 'astro-ph.SR']
492 1 ['astro-ph.GA']
493 6 ['astro-ph.HE']
494 4 ['astro-ph.EP']
495 7 ['astro-ph.CO']
496 6 ['astro-ph.HE']
497 6 ['astro-ph.HE']
498 5 ['astro-ph.SR']
499 7 ['astro-ph.CO']
500 6 ['astro-ph.HE']
501 6 ['astro-ph.HE', 'astro-ph.IM']
502 1 ['astro-ph.GA', 'astro-ph.CO']
503 7 ['astro-ph.CO']
504 7 ['astro-ph.CO']
505 6 ['gr-qc', 'astro-ph.HE']
506 6 ['astro-ph.HE']
507 7 ['astro-ph.CO', 'gr-qc']
508 4 ['astro-ph.EP']
509 1 ['astro-ph.GA', 'astro-ph.CO', 'astro-ph.HE']
510 3 ['astro-ph.IM', 'astro-ph.HE', 'gr-qc']
511 1 ['astro-ph.GA', 'astro-ph.CO', 'astro-ph.HE', 'physics.comp-ph']
512 1 ['astro-ph.GA']
513 7 ['hep-ph', 'astro-ph.CO', 'gr-qc']
514 4 ['astro-ph.EP']
515 1 ['astro-p

1467 4 ['astro-ph.EP']
1468 5 ['gr-qc', 'astro-ph.SR', 'math-ph', 'math.MP']
1469 6 ['astro-ph.HE']
1470 5 ['astro-ph.SR']
1471 7 ['gr-qc', 'astro-ph.CO', 'hep-th']
1472 1 ['astro-ph.GA']
1473 6 ['astro-ph.HE']
1474 1 ['astro-ph.GA', 'astro-ph.IM', 'astro-ph.SR']
1475 6 ['astro-ph.HE']
1476 6 ['astro-ph.HE']
1477 1 ['astro-ph.GA', 'astro-ph.CO']
1478 6 ['physics.plasm-ph', 'astro-ph.HE']
1479 6 ['astro-ph.HE', 'astro-ph.GA']
1480 1 ['astro-ph.GA']
1481 5 ['astro-ph.SR']
1482 3 ['astro-ph.IM']
1483 1 ['astro-ph.GA']
1484 1 ['astro-ph.GA']
1485 7 ['astro-ph.CO', 'astro-ph.IM']
1486 5 ['astro-ph.SR']
1487 1 ['astro-ph.GA']
1488 3 ['astro-ph.IM', 'astro-ph.GA']
1489 6 ['astro-ph.HE']
1490 5 ['astro-ph.SR']
1491 5 ['astro-ph.SR']
1492 1 ['astro-ph.GA', 'astro-ph.CO']
1493 1 ['astro-ph.GA']
1494 6 ['astro-ph.HE']
1495 1 ['astro-ph.GA']
1496 7 ['gr-qc', 'astro-ph.CO', 'hep-th']
1497 1 ['astro-ph.GA']
1498 4 ['astro-ph.EP']
1499 6 ['hep-ph', 'astro-ph.HE', 'hep-ex', 'nucl-ex', 'nucl-th']
1500 

In [109]:
cat = ['astro-ph.CO', 'astro-ph.GA', 'astro-ph.IM']

In [114]:
df['label'].astype('int64')

23      1
31      5
32      1
33      7
34      7
35      1
36      7
37      1
38      6
39      6
40      7
41      5
42      7
43      7
44      5
45      1
46      6
47      7
48      7
49      5
50      1
51      1
52      1
53      1
54      5
55      1
56      5
57      3
58      5
59      1
       ..
1843    1
1844    1
1845    3
1846    1
1847    1
1848    5
1849    5
1850    5
1851    5
1852    6
1853    6
1854    6
1855    1
1856    3
1857    4
1858    4
1859    4
1860    4
1861    7
1862    6
1863    3
1864    3
1865    1
1866    5
1867    7
1868    3
1869    4
1870    4
2615    3
3065    3
Name: label, Length: 1843, dtype: int64

In [27]:
any(['astro-ph.' in c for c in cat])

True

In [29]:
target_name_dict = { 'astro-ph.GA' : 1,
                     'astro-ph.SR' : 2,
                     'astro-ph.IM' : 3,
                     'astro-ph.EP' : 4,
                     'astro-ph.SR' : 5,
                     'astro-ph.HE' : 6,
                     'astro-ph.CO' : 7
                   }

In [33]:
target_name_dict['astro-ph']

KeyError: 'astro-ph'

In [31]:
from sklearn import preprocessing

In [38]:
enc = preprocessing.OneHotEncoder(target_name_dict, sparse=False)

In [40]:
?enc

In [41]:
target_name_dict.keys()

dict_keys(['astro-ph.GA', 'astro-ph.SR', 'astro-ph.IM', 'astro-ph.EP', 'astro-ph.HE', 'astro-ph.CO'])

In [47]:
cat = ['astro-ph.EP', 'astro-ph.SR', 'physics.flu-dyn', 'physics.geo-ph']

for i in range(len(cat)):
    if cat[i] in target_name_dict.keys():
        label = target_name_dict[cat[i]]
        print(label)
        break

4


In [78]:
df.abstract.shape

(1843,)

In [79]:
for i, rows in df.iterrows():
    print(i, rows)

23 title         A note on space dimensionality constraints rel...
abstract      A general sketch of how the problem of space d...
categories             [physics.hist-ph, astro-ph, astro-ph.GA]
created                                     2008-06-03 00:00:00
id                                                    0806.0684
doi                                                        None
label                                                         1
Name: 23, dtype: object
31 title         Seismological constraints on the high-gravity ...
abstract      A seismological study of the pulsating PG1159 ...
categories                                        [astro-ph.SR]
created                                     2009-03-23 00:00:00
id                                                    0903.3912
doi                              10.1088/1742-6596/172/1/012078
label                                                         5
Name: 31, dtype: object
32 title         The kinematics of NGC1333-IRAS2A 

114 title         A Spectroscopic Study of DD UMa: Ursa Major Gr...
abstract      The Ursa Major group is a nearby stellar super...
categories                                        [astro-ph.SR]
created                                     2011-07-25 00:00:00
id                                                    1107.4856
doi                                10.1016/j.newast.2011.07.006
label                                                         5
Name: 114, dtype: object
115 title             Testing the Origin of High-Energy Cosmic Rays
abstract      Recent accurate measurements of cosmic-ray (CR...
categories                                        [astro-ph.HE]
created                                     2011-08-04 00:00:00
id                                                    1108.1023
doi                                  10.1088/0004-637X/752/1/68
label                                                         6
Name: 115, dtype: object
116 title         The puzzling assembly of the

440 title           SDSS-IV eBOSS emission-line galaxy pilot survey
abstract      The Sloan Digital Sky Survey IV extended Baryo...
categories                           [astro-ph.CO, astro-ph.GA]
created                                     2015-09-16 00:00:00
id                                                   1509.05045
doi                                 10.1051/0004-6361/201527377
label                                                         7
Name: 440, dtype: object
441 title         Probing the cosmological viability of non-gaus...
abstract      Based on the relationship between thermodynami...
categories                                 [gr-qc, astro-ph.CO]
created                                     2015-09-16 00:00:00
id                                                   1509.05059
doi                               10.1088/1475-7516/2016/08/051
label                                                         7
Name: 441, dtype: object
442 title         SZ effects in the Magneticum

986 title         Theoretical lower limits on sizes of ultra fai...
abstract      Dwarf spheroidal galaxies are the smallest kno...
categories                      [astro-ph.GA, physics.space-ph]
created                                     2016-05-16 00:00:00
id                                                   1605.04820
doi                                       10.1093/mnras/stw1867
label                                                         1
Name: 986, dtype: object
987 title         Si K Edge Structure and Variability in Galacti...
abstract      We survey the Si K edge structure in various a...
categories                           [astro-ph.GA, astro-ph.HE]
created                                     2016-05-16 00:00:00
id                                                   1605.04837
doi                                  10.3847/0004-637X/827/1/49
label                                                         1
Name: 987, dtype: object
988 title              The near-to-mid infrare

1769 title          The coronal evolution of pre-main-sequence stars
abstract      The bulk of X-ray emission from pre-main-seque...
categories                                        [astro-ph.SR]
created                                     2016-08-18 00:00:00
id                                                   1608.05434
doi                                                        None
label                                                         5
Name: 1769, dtype: object
1770 title         Studying Relation Between Star Formation and M...
abstract      We present $\mathrm{^{12}CO}$ and $\mathrm{^{1...
categories                           [astro-ph.GA, astro-ph.SR]
created                                     2016-08-18 00:00:00
id                                                   1608.05451
doi                                                        None
label                                                         1
Name: 1770, dtype: object
1771 title         Instrumental performanc

In [80]:
rows.shape

(7,)

In [81]:
rows[4]

'astro-ph/0607067'

In [82]:
df.label

23      1.0
31      5.0
32      1.0
33      7.0
34      7.0
35      1.0
36      7.0
37      1.0
38      6.0
39      6.0
40      7.0
41      5.0
42      7.0
43      7.0
44      5.0
45      1.0
46      6.0
47      7.0
48      7.0
49      5.0
50      1.0
51      1.0
52      1.0
53      1.0
54      5.0
55      1.0
56      5.0
57      3.0
58      5.0
59      1.0
       ... 
1843    1.0
1844    1.0
1845    3.0
1846    1.0
1847    1.0
1848    5.0
1849    5.0
1850    5.0
1851    5.0
1852    6.0
1853    6.0
1854    6.0
1855    1.0
1856    3.0
1857    4.0
1858    4.0
1859    4.0
1860    4.0
1861    7.0
1862    6.0
1863    3.0
1864    3.0
1865    1.0
1866    5.0
1867    7.0
1868    3.0
1869    4.0
1870    4.0
2615    3.0
3065    3.0
Name: label, Length: 1843, dtype: float64

In [84]:
df.reindex()

,title,abstract,categories,created,id,doi,label
23,A note on space dimensionality constraints rel...,A general sketch of how the problem of space d...,"[physics.hist-ph, astro-ph, astro-ph.GA]",2008-06-03,0806.0684,None,1.0
31,Seismological constraints on the high-gravity ...,A seismological study of the pulsating PG1159 ...,[astro-ph.SR],2009-03-23,0903.3912,10.1088/1742-6596/172/1/012078,5.0
32,The kinematics of NGC1333-IRAS2A - a true Clas...,Low-mass star formation is described by gravit...,"[astro-ph.GA, astro-ph.SR]",2009-05-28,0905.4575,10.1051/0004-6361/200810831,1.0
33,Optically-passive spirals: The missing link in...,Galaxies migrate from the blue cloud to the re...,"[astro-ph.CO, astro-ph.GA]",2009-06-01,0906.0306,None,7.0
34,Studies of Cosmic Ray Composition and Air Show...,Studies of the composition of the highest ener...,"[astro-ph.CO, astro-ph.HE]",2009-06-11,0906.2319,None,7.0
35,The Super-Alfv\'enic Model of Molecular Clouds...,Recent measurements of the Zeeman effect in da...,[astro-ph.GA],2009-07-03,0907.0587,10.1088/0004-637X/702/1/L37,1.0
36,Rapid Merger of Binary Primordial Black Holes:...,We propose a new scenario for the evolution of...,"[astro-ph.CO, gr-qc]",2009-09-09,0909.1738,10.1093/pasj/psw065,7.0
37,Radial orbit instability: review and perspectives,This paper presents elements about the radial ...,[astro-ph.GA],2009-10-27,0910.5177,10.1080/00411450.2011.654750,1.0
38,Acceleration and radiation of ultra-high energ...,Clusters of galaxies are believed to be capabl...,[astro-ph.HE],2009-10-29,0910.5715,10.1051/0004-6361/200913568,6.0
39,Simultaneous multi-frequency observation of th...,The blazar PG 1553+113 is a well known TeV gam...,"[astro-ph.HE, astro-ph.CO]",2009-11-05,0911.1088,10.1051/0004-6361/200913607,6.0


In [88]:
for i, rows in df.iterrows():
    print(i)

23
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
29

In [89]:
?pd.Series

In [106]:
df.label.get_value(23)

1.0